In [1]:
import re
from html import unescape

# 前節でダウンロードしたファイルを開き、中身を変数htmlに格納する。
with open('dp.html') as f:
    html = f.read()

# re.findall()を使って、書籍1冊に相当する部分のHTMLを取得する。
# *?は*と同様だが、なるべく短い文字列にマッチする（non-greedyである）ことを表すメタ文字。
for partial_html in re.findall(r'<a itemprop="url".*?</ul>\s*</a></li>', html, re.DOTALL):
    # 書籍のURLは itemprop="url" という属性を持つa要素のhref属性から取得する。
    url = re.search(r'<a itemprop="url" href="(.*?)">', partial_html).group(1)
    url = 'https://gihyo.jp' + url  # / で始まっているのでドメイン名などを追加する。

    # 書籍のタイトルは itemprop="name" という属性を持つp要素から取得する。
    title = re.search(r'<p itemprop="name".*?</p>', partial_html).group(0)
    title = title.replace('<br/>', ' ')  # <br/>タグをスペースに置き換える。str.replace()は文字列を置換する。
    title = re.sub(r'<.*?>', '', title)  # タグを取り除く。
    title = unescape(title)  # 文字参照を元に戻す。

    print(url, title)


https://gihyo.jp/dp/ebook/2016/978-4-7741-8336-7 これからはじめるプログラミング 作って覚える基礎の基礎
https://gihyo.jp/dp/ebook/2016/978-4-7741-8337-4 情報処理技術者試験 平成28年度【秋期】情報セキュリティマネジメント パーフェクトラーニング過去問題集
https://gihyo.jp/dp/ebook/2016/978-4-7741-8338-1 大人の自由時間（大人の自由時間mini） 水泳のきれいなカラダをつくる ～スリムな逆三角形になる！ドライランドトレーニング
https://gihyo.jp/dp/ebook/2016/978-4-7741-8346-6 30レッスン 30レッスンで絶対合格！ Microsoft Office Specialist PowerPoint 2013 テキスト＋問題集
https://gihyo.jp/dp/ebook/2016/978-4-7741-8334-3 ゼロからはじめる ゼロからはじめる 海外旅行でスマホ活用 スマートガイド
https://gihyo.jp/dp/ebook/2016/978-4-7741-8335-0 今すぐ使えるかんたんmini 今すぐ使えるかんたんmini CD&DVD 作成超入門 ［Windows 10対応版］
https://gihyo.jp/dp/ebook/2016/978-4-7741-8325-1 大きな字でわかりやすい 大きな字でわかりやすい ワード2016入門
https://gihyo.jp/dp/ebook/2016/978-4-7741-8322-0 大きな字でわかりやすい 大きな字でわかりやすい エクセル2016 入門
https://gihyo.jp/dp/ebook/2016/978-4-7741-8323-7 今すぐ使えるかんたん 今すぐ使えるかんたん ぜったいデキます！ デジカメ写真活用術 ［Windows 10対応版］
https://gihyo.jp/dp/ebook/2016/978-4-7741-8119-6 Software Design 2016年8月号
https://gihyo.jp/dp/ebook/2016/978-4-7741-8333

# XML(RSS)のスクレイピング

In [2]:
!wget http://gihyo.jp/feed/rss2 -O rss2.xml

--2017-09-28 16:35:13--  http://gihyo.jp/feed/rss2
Resolving gihyo.jp... 104.20.34.31, 104.20.33.31, 2400:cb00:2048:1::6814:221f, ...
Connecting to gihyo.jp|104.20.34.31|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘rss2.xml’

rss2.xml                [ <=>                ]  13.23K  --.-KB/s    in 0.005s  

2017-09-28 16:35:14 (2.74 MB/s) - ‘rss2.xml’ saved [13549]



In [3]:
!ls

ch1_2.ipynb              dp                       rss2.xml
ch1_3.ipynb              dp.1                     sample.scraping-book.com
ch1_4.ipynb              dp.html                  yakei_kobe.csv
ch2_4.ipynb              index.html
ch2_5.ipynb              index.html.1


In [9]:
!cat rss2.xml

<?xml version="1.0" encoding="UTF-8" ?>
<rss version="2.0">
<channel>
 <title>gihyo.jp：総合</title>
 <link>http://gihyo.jp/</link>
 <description>gihyo.jp（総合）の更新情報をお届けします</description>
 <language>ja-jp</language>
 <copyright>技術評論社 2017</copyright>
 <lastBuildDate>Thu, 28 Sep 2017 16:22:00 +0900</lastBuildDate>
 <image>
  <url>http://gihyo.jp/assets/templates/gihyojp2007/image/header_logo_gihyo.gif</url>
  <title>gihyo.jp</title>
  <link>http://gihyo.jp/</link>
 </image>
 <item>
  <title>2017年9月28日　Ubuntu，Artful Aardvarkで32ビット版サポートを中止へ ── Linux Daily Topics</title>
  <link>http://gihyo.jp/admin/clip/01/linux_dt/201709/28</link>
  <description>10月19日リリース予定の「Ubuntu 17.10（開発コード: Artful Aardvark）」では，32ビット（i386）アーキテクチャのインストールイメージを含めないことを開発者向けメーリングリストで明らかにした。</description>
  <pubDate>Thu, 28 Sep 2017 16:11:00 +0900</pubDate>
  <category domain="http://gihyo.jp/admin/clip/01/linux_dt">Linux Daily Topics</category>
  <guid>http://gihyo.jp/admin/clip/01/linux_dt/201709/28</guid> 
  <author>階戸アキラ</a

In [4]:
# xml.etree.ElementTree
from xml.etree import ElementTree

tree = ElementTree.parse('rss2.xml')

In [6]:
type(tree)

xml.etree.ElementTree.ElementTree

In [12]:
root = tree.getroot()

In [14]:
root.findall('channel/item')

[<Element 'item' at 0x10ff71098>,
 <Element 'item' at 0x10ff71368>,
 <Element 'item' at 0x10ff71638>,
 <Element 'item' at 0x10ff71958>,
 <Element 'item' at 0x10ff71c78>,
 <Element 'item' at 0x10ff71f48>,
 <Element 'item' at 0x10ff74278>,
 <Element 'item' at 0x10ff74548>,
 <Element 'item' at 0x10ff74818>,
 <Element 'item' at 0x10ff74ae8>,
 <Element 'item' at 0x10ff74db8>,
 <Element 'item' at 0x10ff780e8>,
 <Element 'item' at 0x10ff783b8>,
 <Element 'item' at 0x10ff786d8>,
 <Element 'item' at 0x10ff789f8>,
 <Element 'item' at 0x10ff588b8>,
 <Element 'item' at 0x10ff78c78>,
 <Element 'item' at 0x10ff78f98>,
 <Element 'item' at 0x10ff3b2c8>,
 <Element 'item' at 0x10ff3b598>]

In [18]:
root.findall('channel/item')[0].find('title').text

'2017年9月28日\u3000Ubuntu，Artful Aardvarkで32ビット版サポートを中止へ ── Linux Daily Topics'

In [21]:
for item in root.findall('channel/item'):
    title = item.find('title').text
    url = item.find('link').text
    print(url,title)

http://gihyo.jp/admin/clip/01/linux_dt/201709/28 2017年9月28日　Ubuntu，Artful Aardvarkで32ビット版サポートを中止へ ── Linux Daily Topics
http://gihyo.jp/lifestyle/clip/01/everyday-cat/201709/28 2017年9月28日　ご飯屋さんのくろこ ── 技評ねこ部通信
http://gihyo.jp/news/report/2017/09/2801 HTML5 Conference 2017 基調講演レポート ── レポート
http://gihyo.jp/news/interview/2017/09/2801 はじめにオートメーションありき ―創業者が語るVagrant，Packerを生んだ"HashiCorpのプリンシプル" ── インタビュー
http://gihyo.jp/admin/serial/01/infrasec/0038 第38回　2017年9月～今月は脆弱性祭り～どうしてこんなに出てくるのか～ ── インフラセキュリティの処方箋
http://gihyo.jp/design/clip/01/weekly-web-tech/201709/28 2017年9月第4週号 1位は，実例で学ぶFlexboxの完全ガイド，気になるネタは，GoogleのiOS向けキーボードアプリ「Gboard」，日本語に対応 ── 週刊Webテク通信
http://gihyo.jp/lifestyle/clip/01/awt/201709/28 2017年9月第4週　GoogleがHTCのスマホ事業の一部を11億ドルで買収 ── Android Weekly Topics
http://gihyo.jp/lifestyle/clip/01/everyday-cat/201709/27 2017年9月27日　カギしっぽのくろこ ── 技評ねこ部通信
http://gihyo.jp/admin/clip/01/linux_dt/201709/27 2017年9月27日　Microsoft，Open Source Initiativeのプレミアムスポンサーに ── Linux Daily Topics
http://gihyo.jp/a